In [3]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

In [25]:

target = 'flies2.jpg'
patterns = ['fly.jpg', 'fly1.png', 'fly2.png', 'fly3.png', 'fly4.png']
patterns = ['pattern/' + p for p in patterns]

target_img = cv.imread(target, 0)
target_img_colored = cv.imread(target)

target_dim = (target_img.shape[1], target_img.shape[0])
fly_dim_pixel = (target_dim[0] // 7, target_dim[1] // 5)

print(target_dim)
print(fly_dim_pixel)

patterns_imgs = [ cv.imread(p,0) for p in patterns]

"""resize the images so that they are in the target"""
def resize_for_target(img, target_dim):
    return cv.resize(img, target_dim, interpolation = cv.INTER_AREA)

def resize_with_factor(img, factor):
    dim = (img.shape[1], img.shape[0])
    dim = (int(dim[0] * factor), int(dim[1] * factor))
    return cv.resize(img, dim, interpolation = cv.INTER_AREA)

patterns_imgs = [resize_for_target(p, fly_dim_pixel) for p in patterns_imgs]
    
"""Find the pattern in the target and returns the top-left and bottom right corners if found"""
def find_box_with_pattern(target_img, pattern_img):
    pattern_img_cpy = pattern_img.copy()
    methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR',
            'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']
    meth = methods[0]
    w, h = pattern_img.shape[::-1]
    
    method = eval(meth)
    current_target = target_img.copy()
    result = cv.matchTemplate(current_target, pattern_img_cpy, method)
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)
        
    if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
            
    bottom_right = (top_left[0] + w, top_left[1] + h)
    
    print(str(min_val) + " " + str(max_val))
    
    if (max_val < 7000000):
        return None
    
    return (top_left, bottom_right)


"""find a set of corners matching the patterns in the target"""
def find_for_all_patterns(target_img, patterns):
    corners = []
    for p in patterns:
        corners.append(find_box_with_pattern(target_img, p))
    return corners

def find_for_all_patterns_resize(target_img, patterns):
    corners = []
    factors = [1.5, 1, 0.5]
    for p in patterns:
        for f in factors:
            resized_pattern = resize_with_factor(p, f)
            resized_dim = (resized_pattern.shape[1], resized_pattern.shape[0])
            target_dim = (target_img.shape[1], target_img.shape[0])
            if (resized_dim[0] < target_dim[0] and resized_dim[1] < target_dim[1]):
                corner = find_box_with_pattern(target_img, resized_pattern)
                if (corner is not None):
                    corners.append(corner)
    return corners    

def show_labeled_image_with_corners(target_img, corners):
    
    showed_img = target_img.copy()
    
    for c in corners:
        cv.rectangle(showed_img, c[0], c[1], (0, 0, 255), 2)
        
    cv.imshow('image', showed_img)
    cv.waitKey(0)
    cv.destroyAllWindows()     


(360, 165)
(51, 33)


In [26]:
corners = find_for_all_patterns_resize(target_img, patterns_imgs)
show_labeled_image_with_corners(target_img_colored, corners)
print(len(corners))

-2887737.75 6862151.5
-1798261.125 2920706.5
-387246.25 500378.75
-4201993.5 6899628.0
-1999146.25 3082971.0
-425416.8125 574420.3125
-4713789.0 10947400.0
-2171070.5 4890161.0
-645639.75 871988.3125
-4720723.0 8480065.0
-2471673.5 3787617.0
-474046.15625 712887.0
-3985498.25 8406889.0
-1843208.875 3928892.5
-583585.3125 672900.6875
3


In [ ]:
-5339130.5 9971164.0
-2493280.25 5092399.0
-711466.3125 1087420.875
-5530162.0 11429691.0
-2805763.0 5570730.0
-838670.375 1135935.375
-10501176.0 15898986.0
-5398862.5 9481805.0
-1354308.125 2192995.25
-7610752.0 15590421.0
-3614905.75 6865022.5
-915325.375 1502020.75
-8160000.0 17077074.0
-3606160.0 7953486.5
-1071434.25 1513853.375

In [ ]:

img = cv.imread('flies.jpg',0)
img2 = img.copy()
final_img = img.copy()
template = cv.imread('fly.jpg',0)
w, h = template.shape[::-1]
# All the 6 methods for comparison in a list
methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR',
            'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']
for meth in methods:
    img = img2.copy()
    method = eval(meth)
    # Apply template Matching
    res = cv.matchTemplate(img, template, method)
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
    # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
    if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv.rectangle(img,top_left, bottom_right, (0, 0, 0), 2)
    cv.rectangle(final_img,top_left, bottom_right, (255, 0, 0), 2)
    plt.subplot(121),plt.imshow(final_img)
    #plt.subplot(121), plt.imshow(res,cmap = 'gray')
    plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(img, cmap = 'gray')
    
    plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
    plt.suptitle(meth)
    plt.show()